# Tarea 4: Optimización <br/> CC6204 Deep Learning, Universidad de Chile 

**Fecha de entrega: 17 de octubre de 2019 ([Hoja de respuestas](https://colab.research.google.com/drive/1Wb_4WQZdQyr_ByBPtYmtcDtZYDLkuzrr))**

En esta tarea programarás distintos métodos de inicialización y optimización para redes neuronales que construiste en las tareas anteriores ([Tarea 2](https://colab.research.google.com/drive/1ZnrSZQDhDrmp6mwIxqb_NWlXB77Til7c), [Tarea 3](https://colab.research.google.com/drive/1FA-MGLHphtESlFe4nhxq9hF1tLDZdVF9)). 
Además probarás estos cambios entrenando tu nuevos algoritmos para el problema MNIST.

El material necesario para resolver esta tarea es el siguiente:
* [Video Optimización I: Activaciones, Inicialización de Parámetros, Batch Normalization y SGD con Momentum](https://www.youtube.com/watch?v=4lTYNuU5IKs)
* [Video Optimización II: Nesterov, RMSProp y Adam](https://www.youtube.com/watch?v=WGPTHQuMuR4) 

IMPORTANTE: A menos que se exprese lo contrario, sólo podrás utilizar las clases y funciones en el módulo [`torch`](https://pytorch.org/docs/stable/torch.html).

(por Jorge Pérez, https://github.com/jorgeperezrojas, [@perez](https://twitter.com/perez))

# Parte 1: Inicialización de Xavier/He

En esta parte programarás la [inicialización de Xavier](http://proceedings.mlr.press/v9/glorot10a.html) que considera la inicialización de los pesos según el tamaño de cada capa. En particular para el tensor de parámetros $W^{(i)}$ de dimensiones $(d_{i-1},d_{i})$ (o sea, el que relaciona la salida de la capa $i-1$ y la entrada de la capa $i$), la inicialización de Xavier define cada parámetro $w$ en $W^{(i)}$ como
\begin{equation*}
w := r \sqrt{\frac{1}{d_{i-1}}}
\end{equation*}
donde $r$ es un número aleatorio con distribución normal unitaria (media 0 y varianza 1). Recuerde que para el caso en donde las funciones de activación de la capa $i$ sean `relu` la inicialización debiera llevar un $2$ (también conocida como [inicialización de He](https://arxiv.org/abs/1502.01852)) como se muestra a continuación:
\begin{equation*}
w := r \sqrt{\frac{2}{d_{i-1}}}
\end{equation*}
Agrega al inicializador de la clase de tu red neuronal, un parámetro opcional `init` con el que puedas decidir el tipo de inicialización de parámetros que quieras. En particular, si el valor del parámetros es el string `xavier` entonces se debe implementar la inicialización descrita.

# Parte 2: Descenso de gradiente con momentum

En esta parte implementarás el descenso de gradiente con momentum lo que puede mejorar considerablemente el tiempo de entrenamiento de tu red. 
En clases/video vimos dos formulaciones, una considerando el promedio exponencial movil de los gradientes pasados, y otra como una interpretación física. En esta parte implementarás la segunda. Recuerda que en este caso la idea es incrementar una variable de "velocidad" en la dirección contraria del gradiente (que haría las veces de "aceleración") y usarla para actualizar los parámetros en cada paso del descenso estocástico de gradiente. Recuerda que adicionalmente un (hyper)parámetro $\alpha$ de "fricción" se utiliza para evitar la oscilación en direcciones de gradiente muy pronunciado. En particular, para cada conjunto de parámetros $\theta$, y para cada paso del descenso de gradiente se realiza el siguiente cálculo:
<br>

\begin{eqnarray*}
V_{\partial \theta} & := & \alpha V_{\partial \theta} - \lambda \frac{\partial \mathcal{L}}{\partial \theta} \\
\theta & := & \theta + V_{\partial \theta}
\end{eqnarray*}
<br>
en donde $V_{\partial \theta}$ es un tensor de las mismas dimensiones que los parámetros $\theta$ y que se inicializa como $0$ antes de empezar el entrenamiento.

Modifica la implementación de SGD que ya tenías anteriormente para considerar la fórmula de momentum descrita. Para esto agrega un nuevo parámetro opcional `momentum` con un valor por defecto de 0.9.

In [ ]:
# Tu código debiera continuar así

class SGD():
  def __init__(self, red, lr, momentum=0.9):
    # lo que sea necesario inicializar
    # en este caso debes inicializar la variable de velocidad
    pass
  
  def step():
    # actualiza acá los parámetros a partir de los gradientes
    # y el momentum
    pass

# Parte 3: RMSProp

En esta parte implementarás el algoritmo de RMSProp (Root Mean Square Propagation) que mantiene un promedio móvil exponencial de los cuadrados de los gradientes calculados hasta el momento y modifica las tasas de aprendizaje para cada parámetro dependiendo de ese promedio. En específico, RMSProp usa la siguiente regla de actualización para cada conjunto de parámetros $\theta$:
<br>

\begin{eqnarray*}
S_{\partial \theta} & := & \beta S_{\partial \theta} + (1-\beta)\left(\frac{\partial \mathcal{L}}{\partial \theta}*\frac{\partial \mathcal{L}}{\partial \theta}\right) \\
\theta & := & \theta - \lambda\frac{1}{\sqrt{S_{\partial \theta}}}*\frac{\partial \mathcal{L}}{\partial \theta}
\end{eqnarray*}
<br>

donde $S_{\partial \theta}$ es un tensor de las mismas dimensiones de $\theta$ y se inicializa como $0$ antes de emepzar el entrenamiento.
La operación $*$ representa a una multiplicación punto a punto.

Implementa una nueva clase `RMSProp` que implemente este optimizador. El inicializador debiera tener dos parámetros `lr` para la tasa de aprendizaje ($\lambda$ en las fórmulas de arriba) y `beta`, con valores por defecto 0.001 y 0.9, respectivamente. Agrega un parámetro adicional `epsilon` con valor por defecto $10^{-8}$, y úsalo para evitar la división por $0$ sumándolo al denominador en la fórmula de arriba.

In [ ]:
# Tu código acá

class RMSProp():
  def __init__(self, red, lr=0.001, beta=0.9, epsilon=1e-8):
    # en este caso debes inicializar la variable que acumula
    # el promedio exponencial de los cuadrados
    pass
  
  def step():
    # actualiza acá los parámetros a partir de los gradientes
    # y la corrección según S
    pass

# Parte 4: Adam

En esta parte implementarás el algoritmo Adam (Adaptive Moments). Adam calcula un promedio exponencial móvil de los valores previos del gradiente y de los cuadrados del gradiente. El promedio de los gradientes lo usa para el paso del descenso del gradiente (como el SGD con momentum) y el promedio de los cuadrados de los gradientes para modificar la tasa de aprendizaje para cada parámetro por separado. Además Adam utiliza una corrección del sesgo inicial para considerar el hecho de que antes del entrenamiento todos los valores del gradiente (y el cuadrado) comienzan como 0. En específico, las actualizaciones de los parámetros según Adam están dadas por:
<br>

\begin{eqnarray*}
P_{\partial \theta} & := & \beta_1P_{\partial \theta} + (1-\beta_1)\frac{\partial \mathcal{L}}{\partial \theta} \\
S_{\partial \theta} & := & \beta_2 S_{\partial \theta} + (1-\beta_2)\left(\frac{\partial \mathcal{L}}{\partial \theta}*\frac{\partial \mathcal{L}}{\partial \theta}\right) \\
\overline{P_{\partial \theta}} & := & \frac{P_{\partial \theta}}{1-{\beta_1}^n} \\
\overline{S_{\partial \theta}} & := & \frac{S_{\partial \theta}}{1-{\beta_2}^n} \\
\theta & := & \theta - \lambda\frac{1}{\sqrt{\overline{S_{\partial \theta}}}}*\overline{P_{\partial \theta}}
\end{eqnarray*}
<br>

donde $P_{\partial \theta}$ y $S_{\partial \theta}$ son tensores de las mismas dimensiones que $\theta$ y se inicializan como $0$ antes de comenzar el entrenamiento. Adicionalmente $n$ indica el paso de la iteración de descenso de gradiente, lo que se usa para corregir el sesgo inicial cuando se está computando $\overline{P_{\partial \theta}}$ y $\overline{S_{\partial \theta}}$ y la operación $*$ representa a una multiplicación punto a punto.

Implementa una nueva clase `Adam` que implemente este optimizador. El inicializador debiera tener dos parámetros `lr` para la tasa de aprendizaje y `beta1` y `beta2`, con valores por defecto 0.001, 0.9, y 0.999, respectivamente. Agrega un parámetro adicional `epsilon` con valor por defecto $10^{-8}$, y úsalo para evitar la división por $0$ sumándolo al denominador en la fórmula de arriba.

In [ ]:
# Tu código acá

class Adam():
  def __init__(self, red, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
    # en este caso debes inicializar las variable necesarias
    pass
  
  def step():
    # actualiza acá los parámetros a partir de los gradientes
    # y las correcciones
    pass

# Parte 5: Batch Normalization

En esta parte programarás la técnica de Batch Normalization (BN). La idea de BN es agregar una nueva capa entre dos capas de la red, digamos las capas $i$ e $i+1$. Esta nueva capa normaliza los valores de salida de la capa $i$ antes de pasarlas como inputs a la capa $i+1$. 
Al mismo tiempo, cada capa de BN agrega dos conjuntos de parámetros entrenables que le permiten a la red deshacer la aplicación de la normalización si es que esto ayudara al aprendizaje. 
Más formalmente, supongamos que a una capa BN le pasamos como input la salida $h^{(i)}$ de una capa escondida de nuestra red de dimensiones $(B,d)$, donde $B$ representa a la dimensión del batch de ejemplos. Entonces BN calcula el siguiente valor:
<br>

\begin{eqnarray*}
\tilde{h}^{(i)} := \gamma^{(i)}*\bar{h}^{(i)} + \beta^{(i)}
\end{eqnarray*}
<br>
donde $\gamma^{(i)}$ y $\beta^{(i)}$ son tensores de parámetros entrenables de dimensión $(d)$ y $\bar{h}^{(i)}$ es la normalización de $h^{(i)}$ respecto a la dimensión del batch. O sea, $\bar{h}^{(i)}$ se calcula como:
<br>


\begin{equation*}
\bar{h}^{(i)} := \frac{h^{(i)}-\mu^{(i)}}{\sqrt{v^{(i)}}}
\end{equation*}
<br>
Donde $\mu^{(i)}$ es un vector de dimensión $(d)$ con la media de los valores en $h^{(i)}$ (con respecto a la dimensión del batch), y $v^{(i)}$ es un vector también de dimensión $(d)$ con la varianza de los valores en $h^{(i)}$ (con respecto a la dimensión del batch). Nota que las operaciones que generan $\bar{h}^{(i)}$ usan broadcasting.

Luego de la capa BN lo que se pasa a la siguiente capa de la red es $\tilde{h}^{(i)}$ en vez de $h^{(i)}$. O sea, el resultado de la siguiente capa se calcula como
<br>

\begin{equation*}
h^{(i+1)} := f^{(i+1)}(\tilde{h}^{(i)}W^{(i+1)}+b^{(i+1)}).
\end{equation*}
<br>

Nota que durante el backpropagation deberás considerar la nueva capa. En particular, el gradiente con respecto a $h^{(i)}$ ahora pasará por $\tilde{h}^{(i)}$ por lo que deberás calcular 
<br>

\begin{equation*}
\frac{\partial \mathcal{L}}{\partial h^{(i)}} = \frac{\partial \mathcal{L}}{\partial \tilde{h}^{(i)}} \cdot \frac{\partial \tilde{h}^{(i)}}{\partial {h}^{(i)}}.
\end{equation*}
<br>
También deberás calcular los gradientes de $\mathcal L$ con respecto a $\gamma^{(i)}$ y a $\beta^{(i)}$ para poder actualizar esos parámetros. 

Otro aspecto importante de Batch Normalization es que en tiempo de test si le pasamos un único $x$ a la red para predecir (no un batch), no tendremos suficientes datos para caluclar $\mu^{(i)}$ ni $v^{(i)}$. Para esto se mantiene un promedio de los valores de $\mu^{(i)}$ y $v^{(i)}$ computados durante el entrenamiento (en general un promedio exponencial móvil) y se usan estos valores en tiempo de test.

Implementa BN en tu red neuronal. 
Para esto agrega una nueva lista opcional `bn` de valores booleanos a tu red, de manera tal que el valor correspondiente sea `True` si es que si quiere implementar BN después de la capa correspondiente. Por ejemplo, el siguiente código
```
red = FFNN(300, [50,30], [relu,sig], 10, bn=[True,False])
```
crea una red de dos capas escondidas, en donde se aplicará BN después de la primera capa escondida (y no después de la segunda capa escondida).
Ten en cuenta que la función `forward` de tu red debe comportarse de manera distinta mientras entrenas y mientras predices. Para esto debes usar la opción `predict` que ya le agregaste al `forward` de tu red en la tarea de regularización.


In [ ]:
# Tu código debiera continuar como sigue

class FFNN():
  def __init__(self, F, l_h, l_a, C, wc_par=None, keep_prob=None, bn=None):
    # debes crear los parámetros necesarios para las capas de
    # batch normalizacion
    pass
  
  def forward(x, predict=False):
    # debes modificar esta función para considerar las capas para las que se
    # usará batch normalization
    # también debes preocuparte de guardar los datos estadísticos que se
    # usaran en tiempo de test (predict=True)
    pass
  
  def backward(x,y,y_pred):
    # computar acá todos los gradientes considerando las capas de 
    # batch normalization
    # no olvides considerar los nuevos parámetros entrenables.
    pass

# Parte 6: Entrenamiento en MNIST 

Usa tu red neuronal para entrenar con los datos de MNIST y compara como cambia las curvas de entrenamiento dependiendo de factores como la inicialización y los algoritmos que utilices. Presenta al menos dos gráficos en donde compares. Por ejemplo, puedes presentar uno que para la misma estrategia de inicialización, los tres algoritmos de optimización para varias épocas y cómo evoluciona la pérdida y el acierto. Presenta otro para mostrar el mismo efecto pero con y sin BN. En cada caso comenta que conclusiones puedes sacar. Algunos ejemplos de preguntas que podrías tratar de responder son:
* ¿cómo afecta el algoritmo de optimización al tiempo de convergencia de la red?
* ¿cómo afecta el algoritmo de optimización en el acierto alcanzado por la red?
* ¿aplicar, o no, BN afecta a todos los algoritmos de optimización por igual?